In [7]:
# All the basic necessary libraries for geneartion of music 
import sys
import time
import random
import glob
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam, RMSprop
from midi_parser import *

In [8]:
# Declaration of global parameters

x_len = 100 # Length of the sample sequence

y_len = 10  # Length of the output sequence. This value must be close/consistent with the value mentioned in trained.ipynb.

iteration = 50 # determines the number of iteration to generate in new sequence. Final result of total(length): y_len * itertaion


In [10]:
saved_model = "./model/Model_LSTM.json"        # Extracting the path for the json file for LSTM network.
saved_weights = "./model/Weight_LSTM.hdf5"     # Extracting the path for the weight of model of LSTM.
output_folder = "./Muisc_Generated"                        # Make a directory "Music_Generated for storing all the music files"
print("Enter the desired Genre of Music  = ")  # Ask the user to take input for the genre category for which music to be generated
sample_folder = input()

C:/Users/shivam/Documents/Music_Generator_Phase2/MuiscGeneration/Classical


In [11]:
print("Enter the name to save your music = ") # Enter the desired name for the name of the song which you want to save.
sav_name = input()

Enter the name to save your music = 
newClassical.mid


In [12]:
# Function for generating music from the category of music inserted with adjusting in tempo and its resolution.

def generate(input_data, tempo, resolution):
    
    '''
        Generate the new music and save to a midi file folder with the desired name given by the user.

        parameters and some terminologies:
        
                input_data: fetch music pianoroll for music generation.
                tempo: tempo value parsed from the fetched music after analysing it.
                resolution: resolution value parsed from the fetched music. It is nothing but the move of a note or chord.
    '''

    output_path = os.path.join(output_folder,sav_name) # Assign where to store our all music generated files.

    # This part randomly select a sequence from the seed music
    start = np.random.randint(0, input_data.shape[0]-1-x_len-iteration)
    pattern = np.array(input_data[start:start+x_len])
    
    # Making a prediction output array where all the chord notes will be stored.
    prediction_output = [] 

    # concatenate all generated sequence
    for i in range(iteration):
        
        prediction = model.predict(pattern.reshape(1,pattern.shape[0],-1).astype(float)).reshape(y_length,-1) # generating the desired sequence
        prediction_output.append(prediction)
        pattern = np.append(pattern[y_length:,], prediction, axis=0) # shift sliding window on input data

    print("output shape of the array: ", np.array(prediction_output).shape)

    # convert sequence back to piano roll
    pianoroll = outputPianoRoll(np.array(prediction_output), note_threshold=0.1)
    print("pianoroll shape: ", pianoroll.shape)
    
    # convert piano roll back to midi format of music
    outputMidi(output_path, pianoroll, tempo, resolution, scale=int(y_len))

    
# Now below part is where we are loading our trained model which is trained by the trained.ipynb file.

if __name__ == '__main__':

    # load trained model      
    
    model = model_from_json(open(saved_model).read()) # Loading json file
    
    model.load_weights(saved_weights)                 # Loading our model weight
    
    model.summary()                                   # Generate the summary of the model
    
    model.compile(loss='categorical_crossentropy', optimizer = RMSprop())
    

    # randomly select a file from sample folder
    midi_files = [file for file in os.listdir(sample_folder) if file.endswith(".midi") or file.endswith(".mid")]
    input_data, tempo, resolution = parseMidi( os.path.join(sample_folder, midi_files[random.randint(0,len(midi_files)-1)]) )

    # generate new music
    generate(input_data, tempo, resolution)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, None, 512)         1150976   
_________________________________________________________________
batch_normalization_5 (Batch (None, None, 512)         2048      
_________________________________________________________________
dropout_20 (Dropout)         (None, None, 512)         0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 10, 512)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 10, 512)           2099200   
_________________________________________________________________
dropout_21 (Dropout)         (None, 10, 512)          